In [311]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    model_name="llama-3.1-8b-instant",
    api_key="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",
)

In [312]:
import time
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate(
    template="""
    <|begin_of_text|>
    <|start_header_id|>system<|end_header_id|>  
    You are an expert at identifying table of contents from the given paragraph {paragraph} 
    Don't give any additional information or summary
    You must add the sub category but don't add 'The Official Gazette' and which edition

    ### The output must be in the following format: 
    List of dictonary, each dictonary key should be main topics like Executive Council Resolutions, Laws and value should be list of sub topics
    Do not include Pages

    Sample output format
    "Laws": [
            "Law No. (2) of 2022 Concerning the Reorganisation of the National Rehabilitation Centre - Abu Dhabi"
        ],
        "Executive Council Resolutions": [
            "Chairman of the Executive Council Resolution No. (3) of 2022 Concerning the Appointment of the Director-General for Tourism at the Department of Culture and Tourism"
        ],
        "Circulars": [
            "Circular No. (1) of 2022 Concerning the Policy of Providing an Easy and Effortless Customer Experience in the Emirate of Abu Dhabi"
        ]

    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["paragraph"],
)

question_router = prompt | llm


In [313]:
from PyPDF2 import PdfReader
import re
pdf_path = "../doc/8th Edition English 2023.pdf"

def table_of_content(reader):
    for page_no in range(1, 4):
        page = reader.pages[page_no]
        text = page.extract_text()
        return text
        

with open(pdf_path, 'rb') as file:
    reader = PdfReader(file)

    num_pages = len(reader.pages)

    index_text = table_of_content(reader)
    index_data = question_router.invoke({"paragraph": index_text})
    output = ""
    for page_no in range(3, num_pages):
        page = reader.pages[page_no]
        text = page.extract_text()
        if text and text[0].isdigit():
            text = text[1:]
            text = re.sub(r'^\d+', '', text)

        cleaned_text = re.sub(r'[\n\r\t]', ' ', text)
        cleaned_text = cleaned_text.replace("  ", " ")
        cleaned_text = cleaned_text.replace("  ", " ")


        output = output + " " + cleaned_text
        output = output.strip()

In [314]:
print(index_data.content)

{
    "Laws": [
        "Law No. (12) of 2023 concerning the Punitive and Correctional Establishments in the Emirate of Abu Dhabi"
    ],
    "Chairman of the Executive Council Resolutions": [
        "Chairman of the Executive Council Resolution No. (4) of 2023 Concerning the Reformation of the Board of Directors of The Emirates Foundation",
        "Chairman of the Executive Council Resolution No. (5) of 2023 Concerning the Repeal of the Chairman of the Executive Council Resolution No. (65) of 2014 concerning scholarships and study leaves in the Emirate of Abu Dhabi",
        "Chairman of the Executive Council Resolution No. (6) of 2023 Concerning the Reformation of the Board of Directors of the Advanced Technology Research Council"
    ],
    "Executive Council Resolutions": [
        "Executive Council Resolution No. (165) of 2023 concerning The Appointment of the Director-General of the International Fund for Houbara Conservation",
        "Executive Council Resolution No. (185) o

In [315]:
output

'Laws Laws Law No. (12) of 2023 concerning the Punitive and Correctional Establishments in the Emirate of Abu Dhabi We, Mohamed bin Zayed Al Nahyan, Ruler of Abu Dhabi, Having reviewed: • Law No. (1) of 1974 concerning the reorganisation of the Governmental body in the Emirate of Abu Dhabi and its amendments; • Law no. (23) of 2006 concerning the Judicial Department in the Emirate of Abu Dhabi, and its amendments; • Law No. (1) of 2020 concerning Abu Dhabi Police General Headquarters and its amendments; • Federal Law no. (43) of 1992 concerning the Regulation of Punitive Establishments; • Federal Law by Decree No. (31) of 2021 concerning the issuance of the Crimes and Penal Law, and its amendments; • Federal Law No. (6) of 2022 concerning Juveniles, Delinquents, and those at risk of Delinquency; • Federal Law by Decree No. (38) of 2022 concerning the issuance of the Criminal Procedures Law; and • Based on what was presented to and approved by the Executive Council; Issued the following

In [316]:
import json
TOC = json.loads(index_data.content)
key_topics = list()

final_dict = {}

for key, value in TOC.items():
    key_topics.extend(value)

key_topics

['Law No. (12) of 2023 concerning the Punitive and Correctional Establishments in the Emirate of Abu Dhabi',
 'Chairman of the Executive Council Resolution No. (4) of 2023 Concerning the Reformation of the Board of Directors of The Emirates Foundation',
 'Chairman of the Executive Council Resolution No. (5) of 2023 Concerning the Repeal of the Chairman of the Executive Council Resolution No. (65) of 2014 concerning scholarships and study leaves in the Emirate of Abu Dhabi',
 'Chairman of the Executive Council Resolution No. (6) of 2023 Concerning the Reformation of the Board of Directors of the Advanced Technology Research Council',
 'Executive Council Resolution No. (165) of 2023 concerning The Appointment of the Director-General of the International Fund for Houbara Conservation',
 'Executive Council Resolution No. (185) of 2023 concerning the Amendment of the Membership of the Board of Directors of the Environment Agency - Abu Dhabi',
 'Executive Council Resolution No. (187) of 2023

In [309]:
# output_lower = output.lower()

# def remove_dot(text):
#     if text.endswith('.'):
#         text = text[:-1]
#     else:
#         text = text
#     return text

# for i in range(len(key_topics) - 1):

#     current_topic = key_topics[i][:-1]
#     next_topic = key_topics[i + 1]

#     current_topic = remove_dot(current_topic)
#     next_topic = remove_dot(next_topic)

#     current_topic_lower = current_topic.lower()
#     next_topic_lower = next_topic.lower()

#     print("===========================================")

#     current_topic = current_topic_lower.replace("'", '"')
#     next_topic = next_topic_lower.replace("'", '"')
    
#     print(f"Current Topic: {current_topic}")
#     print(f"Next Topic: {next_topic}")

#     start = output_lower.find(current_topic)
#     end = output_lower.find(next_topic)
#     print(f"String index of current topic: {start}")
#     print(f"String index of next topic: {end}")

#     substring = output_lower[start:end]
#     print("\n")
#     print(substring)

In [317]:
output_lower = output.lower()

def remove_dot(text):
    return text[:-1] if text.endswith('.') else text

for i in range(len(key_topics)):
    current_topic = key_topics[i]
    
    current_topic = remove_dot(current_topic)
    current_topic_lower = current_topic.lower()

    print("===========================================")
    current_topic = current_topic_lower.replace("'", '"')
    
    print(f"Current Topic: {current_topic}")

    if i < len(key_topics) - 1:
        next_topic = key_topics[i + 1]
        next_topic = remove_dot(next_topic)
        next_topic_lower = next_topic.lower()
        
        next_topic = next_topic_lower.replace("'", '"')
        print(f"Next Topic: {next_topic}")

        start = output_lower.find(current_topic)
        end = output_lower.find(next_topic)

        if start == -1:
            start = end - 1000
        elif end == -1:
            end = start + 1000
        elif start == -1 and end == -1:
            break
            
        print(f"String index of current topic: {start}")
        print(f"String index of next topic: {end}")

        substring = output_lower[start:end]
    else:
        # For the last topic, take everything from the current topic to the end
        start = output_lower.find(current_topic)
        end = len(output_lower)  # Go to the end of the string
        substring = output_lower[start:end]

    print("\n")
    print(substring)


Current Topic: law no. (12) of 2023 concerning the punitive and correctional establishments in the emirate of abu dhabi
Next Topic: chairman of the executive council resolution no. (4) of 2023 concerning the reformation of the board of directors of the emirates foundation
String index of current topic: 10
String index of next topic: 1010


law no. (12) of 2023 concerning the punitive and correctional establishments in the emirate of abu dhabi we, mohamed bin zayed al nahyan, ruler of abu dhabi, having reviewed: • law no. (1) of 1974 concerning the reorganisation of the governmental body in the emirate of abu dhabi and its amendments; • law no. (23) of 2006 concerning the judicial department in the emirate of abu dhabi, and its amendments; • law no. (1) of 2020 concerning abu dhabi police general headquarters and its amendments; • federal law no. (43) of 1992 concerning the regulation of punitive establishments; • federal law by decree no. (31) of 2021 concerning the issuance of the cri

In [242]:
output_lower

'laws laws law no. (1) of 2021 concerning the amendment of some provisions of law no. (5) of 1996 concerning al ain farms for livestock production  we, khalifa bin zayed al nahyan, ruler of abu dhabi having reviewed: • law no. (1) of 1974 concerning the reorganisation of the governmental body in the emirate of abu dhabi and its amendments; • law no. (2) of 1971 concerning the national advisory council, and its amendments; • law no. (5) of 1996 concerning al ain farms for livestock production, and its amendments; • law no. (10) of 1999 concerning the establishment of abu dhabi fund for development, and its amendments; and • based on what was presented to and approved by the executive council; issued the following law: article (1) article (2) of aforementioned law no. (5) of 1996 shall be replaced by the following provisions: “the capital of the farms is set at (aed 510,204,000), divided into (5,102,040) shares and the value of each share is (aed 100). the ownership of the shares of the 

In [ ]:
text = "law no. (1) of 2021 concerning the amendment of some provisions of law no. (5) of 1996 concerning al ain farms for livestock production."

if text.endswith('.'):
    text = text[i][:-1]
else:
    current_topic = key_topics[i]

In [ ]:
output_lower = output.lower()

def remove_dot(text):
    if text.endswith('.'):
        text = text[:-1]
    else:
        text = text
    return text

for i in range(len(key_topics) - 1):  # Loop up to the second-to-last element

    current_topic = remove_dot(current_topic)
    next_topic = remove_dot(next_topic)

    current_topic_lower = current_topic.lower()
    next_topic_lower = next_topic.lower()

    print("===========================================")

    current_topic = current_topic_lower.replace("'", '"')
    next_topic = next_topic_lower.replace("'", '"')
    
    print(f"Current Topic: {current_topic}")
    print(f"Next Topic: {next_topic}")

    start = output_lower.find(current_topic)
    end = output_lower.find(next_topic)
    print(f"String index of current topic: {start}")
    print(f"String index of next topic: {end}")

    substring = output_lower[start:end]
    print("\n")
    print(substring)